In [25]:
import pandas as pd
import numpy as np
from langdetect import detect
import re 
import nltk
from nltk.corpus import stopwords
import unicodedata
from rapidfuzz import process, fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import requests
import json
import fitz

In [2]:
# Importing data 
cards = pd.read_csv('cards_trello.csv')
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado']
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado']


Preprocessing data 

In [3]:
cards['desc_lang'] = [detect(str(x)) for x in cards['desc']]

In [6]:
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],pt
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],pt


In [4]:
cards['desc_lang'].value_counts()

desc_lang
pt    225
en     93
tl      4
es      1
Name: count, dtype: int64

In [4]:
replace_values = {'pt' : 'portuguese', 'en' : 'english', 'tl' : 'portuguese', 'es': 'spanish'}

cards['desc_lang'] = cards['desc_lang'].replace(replace_values)

In [5]:
def substituir_sinonimos(texto):
    texto = texto.lower()
    texto = texto.replace('business intelligence', 'bi')
    texto = texto.replace('inteligência de negócios', 'bi')
    return texto

def cleaning_soft(text):

    text = str(text)
    text = text.lower()
    text = re.sub(r'[^\w\s-]', '', text)
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = substituir_sinonimos(text)

    return text.strip()



def cleaning_text(text, lang):
    stopwords_lang = set(stopwords.words(lang))

    text = cleaning_soft(text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('about the job', '', text)
    text = re.sub('job description', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [p for p in words if p not in stopwords_lang]

    return ' '.join(words)



In [6]:
cards['desc_clean'] = [cleaning_text(text, lang) for text, lang in zip(cards['desc'], cards['desc_lang'])]

In [7]:
cards['name_clean'] = [cleaning_soft(text) for text in cards['name']]


In [8]:
cards[['vaga', 'empresa', 'fonte']] = cards['name_clean'].str.split('-', n = 2, expand= True)
cards['vaga'] = [re.sub(r'\b(junior|pleno|pl|senior|jr)\b', '', text, flags=re.IGNORECASE) for text in cards['vaga']]
cards['vaga'] = cards['vaga'].apply(cleaning_soft)
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin


In [9]:
vagas_padrao = ['analista de dados', 'cientista de dados', 'analista de bi', 'data analyst',
                'data scientist', 'bi analyst', 'product analyst', 'analista atuarial', 'estatistico', 'especialista bi']

replace_names = {
    'data scientist': 'cientista de dados',
    'data analyst': 'analista de dados',
    'bi analyst': 'analista de bi',
    'especialista bi': 'analista de bi',
    'trainee engenheiro de dta analytics ai': 'engenheiro de dados'
}

In [10]:

# Fallback: verifica se todos os tokens da vaga padrão estão presentes na vaga suja
def fallback_match(vaga_suja, lista_padrao):
    vaga_tokens = set(vaga_suja.lower().split())
    for vaga_padrao in lista_padrao:
        tokens_padrao = set(vaga_padrao.lower().split())
        if tokens_padrao.issubset(vaga_tokens):
            return vaga_padrao
    return None

# Função principal
def padronizar_vaga(vaga_suja, lista_padrao,replace_names,  limiar=70):
    vaga_original = vaga_suja  # guardar original caso nada funcione
    vaga_suja  = str(vaga_suja)
    vaga_suja = re.sub('data science', 'data scientist', vaga_suja)
    vaga_suja = vaga_suja.lower().strip()

    # 1. Fuzzy matching
    match, score, _ = process.extractOne(
        vaga_suja,
        lista_padrao,
        scorer=fuzz.token_set_ratio
    )
    if score >= limiar:
        resultado = match
    else:
        # 2. Verificação por inclusão de tokens
        match_fallback = fallback_match(vaga_suja, lista_padrao)
        resultado = match_fallback if match_fallback else vaga_suja

    # 3. Correções finais com replace_names
    for termo, substituto in replace_names.items():
        if termo in resultado:
            resultado = resultado.replace(termo, substituto)

    return resultado


In [11]:
cards['vaga_standard'] =  [padronizar_vaga(vaga_suja, vagas_padrao, replace_names) for vaga_suja in cards['vaga'] ]

In [12]:
cards['vaga_standard'].drop_duplicates().values

array(['cientista de dados', 'analista de dados', 'fbs', 'analista de bi',
       'product analyst', 'oracle technical ai', 'analista atuarial',
       'alin', 'r programming expert contractor', 'estatistico',
       'engenheiro de dados'], dtype=object)

In [14]:
fonte_padrao = ['linkedin', 'indeed', 'vagas', 'catho', 'glassdoor', 'infojobs']
replace_fonte = {'mercado livre' : 'mercado livre'}

In [15]:
cards['fonte_standard'] = [padronizar_vaga(fonte_suja, fonte_padrao, replace_fonte) for fonte_suja in cards['fonte']]

In [16]:
cards['fonte_standard'].value_counts()

fonte_standard
linkedin           222
indeed              42
catho               25
none                 9
glassdoor            6
hiring cafe          5
infojobs             4
trabalha brasil      3
vagas                2
bairesdev site       1
whatsapp             1
team                 1
mercado livre        1
braintrust           1
Name: count, dtype: int64

Using TF-IDF to get word frequencies

In [ ]:
# Cleaning desc 



In [16]:
corpus = cards['desc_clean'].dropna()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

palavras = vectorizer.get_feature_names_out()
pesos = X.toarray().sum(axis = 0)

desc_pesos = pd.DataFrame({'palavra': palavras, 'peso_tfidf': pesos}).sort_values('peso_tfidf', ascending=False)

In [18]:
desc_pesos.query("peso_tfidf > 2")

,palavra,peso_tfidf
2804,dados,35.079865
2835,data,19.778478
4234,experiencia,12.381810
2264,conhecimento,9.831421
11121,voce,9.365211
...,...,...
3084,desempenho,2.022012
1594,carreira,2.014775
5470,implementar,2.012330
4855,gente,2.009643


In [17]:
# Bag of words
vectorizer_bag = CountVectorizer()
X = vectorizer_bag.fit_transform(corpus)

palavras = vectorizer_bag.get_feature_names_out()
contagens = X.toarray().sum(axis = 0)

desc_bag = pd.DataFrame({'palavra': palavras, 'frequência': contagens}).sort_values('frequência', ascending=False)
desc_bag

,palavra,frequência
2804,dados,1644
2835,data,1189
4234,experiencia,526
2264,conhecimento,417
11121,voce,388
...,...,...
5346,hypergrowth,1
5344,hype,1
5343,hyderabad,1
5341,hungry,1


In [140]:
# Using Ollama to extrat job atributes 

OLLAMA_MODEL = 'llama3.2:latest'

def extrair_requisitos(texto_vaga):
    prompt = f"""
A partir da descrição da vaga abaixo, retorne um JSON com os seguintes campos:

- "tecnicas": lista das habilidades técnicas
- "comportamentais": lista das habilidades comportamentais
- "nivel": "junior", "pleno" ou "senior"

Descrição:
\"\"\"{texto_vaga}\"\"\"

Responda apenas com JSON válido.
"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
    )

    if response.status_code == 200:
        raw = response.json()["response"].strip()
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            print("⚠️ Erro ao interpretar como JSON. Conteúdo bruto:")
            print(raw)
            return None
    else:
        print("⚠️ Erro na requisição:", response.status_code)
        return None

In [143]:
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte,vaga_standard,fonte_standard
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin,cientista de dados,linkedin
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin,analista de dados,linkedin


In [144]:
# Suponha que seu DataFrame é `cards` com colunas: id_vaga, desc_clean

resultados = []

for _, row in cards.iterrows():
    requisitos = extrair_requisitos(row['desc_clean'])
    if requisitos:
        nivel = requisitos.get('nivel')
        for tipo, habilidades in [('tecnica', requisitos.get('tecnicas', [])),
                                  ('comportamental', requisitos.get('comportamentais', []))]:
            for habilidade in habilidades:
                resultados.append({
                    'id_vaga': row['id'],
                    'tipo_habilidade': tipo,
                    'habilidade': habilidade
                })
        # adicionar o nível ao DataFrame principal
        cards.loc[cards['id'] == row['id'], 'nivel_estimado'] = nivel


⚠️ Erro ao interpretar como JSON. Conteúdo bruto:
```json
{
    "tecnicas": [
        "estatistica",
        "matematica aplicada",
        "economia",
        "ciencia computacao",
        "tecnologia",
        "fisica",
        "programacao (ex: SQL, Python, R, PySpark, Spark)",
        "Power BI"
    ],
    "comportamentais": [
        "comunicar descobertas de forma clara e compreensível",
        "manter dashboards e relatórios atualizados",
        "buscar maneiras de aprimorar a eficiência",
        "trabalhar em equipe para alcançar objetivos",
        "apresentar resultados de forma clara e concisa"
    ],
    "nivel": "junior"
}
```
⚠️ Erro ao interpretar como JSON. Conteúdo bruto:
```
{
  "tecnicas": [
    "Linguagens de programação",
    "Power BI",
    "Excel",
    "Python",
    "Knime",
    "Ferramentas específicas"
  ],
  "comportamentais": [
    "Colaboração em equipe",
    "Trabalho independente",
    "Comunicação eficaz",
    "Resolução de problemas",
    "Gerenciamen

In [126]:
cards['desc_clean'][0]

'time analytics insights busca profissional habilidades analise dados interpretacao metricas geracao insights estrategicos principal responsabilidade transformar grandes volumes dados informacoes objetivas identificando padroes tendencias oportunidades melhoria tambem papel equipe desenvolver manter dashboards relatorios analises preditivas sempre buscando aprimorar eficiencia responsibilities and assignments criacao relatorios dashboards personalizados times estrategicos operacionais area negocio porto bank proporcionando suporte compreensao metricas jornada processo coleta enriquecimento estruturacao bases dados utilizadas construcao relatorios utilizando sistemas disponiveis companhia atraves programacao interpretacao resultados analises geracao insights valiosos comunicando descobertas atraves apresentacoes dashboards relatorios maneira clara compreensivel todos stakeholders acompanhamento continuo relatorios existentes assegurar qualidade informacoes manutencao frequencia adequada

In [151]:
resultados
resultados_df = pd.json_normalize(resultados)
resultados_df.head(2)

,id_vaga,tipo_habilidade,habilidade
0,68513c86c8825584ffc53bd9,tecnica,SQL
1,68513c86c8825584ffc53bd9,tecnica,Excel


In [153]:
resultados_df.to_csv("vaga_requisitos.csv", index = False)

Get my curriculum in PDf and compare to th ejob positions

In [26]:
def get_text_pdf(file):
    doc = fitz.open(file)
    texto_en = "" 
    for page in doc:
        texto_en += page.get_text()
    return texto_en

In [27]:
cv_en = get_text_pdf("ESMB170725.pdf")
cv_pt = get_text_pdf("Erika Borelli CV220725.pdf")

AttributeError: module 'fitz' has no attribute 'open'

In [ ]:
fitz.op

In [28]:
import fitz
print(fitz.__doc__)  # pode ser None, não tem problema
print(dir(fitz))     # veja se aparece 'open'


None
['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']
